In [8]:
import pandas as pd 
import numpy as np
import re

from pymongo import MongoClient

In [71]:
from zhtools import langconv

## 导入数据

In [9]:
# 从mongo数据库中读取数据
client = MongoClient()
db = client['douban']
table = db['world_film']
df_all = pd.DataFrame(list(table.find()))

In [10]:
df_all.shape

(126711, 13)

In [11]:
df_all.head(1)

,_id,actor,date,director,id,language,rate,rating_num,region,runtime,title,type,year
0,5c024433f625b7c00bfbe877,"[扎克·埃夫隆, 莱斯利·曼恩, 托马斯·列农, 马修·派瑞, Tyler Steelman...",[2009-04-17(美国)],[布尔·斯蒂尔斯],2213591,[英语],7.3,155828,[美国],[102 分钟],重返十七岁 17 Again,"[剧情, 喜剧, 爱情]",2009


## 数据清洗

### 将各列的空列表转为空值

In [12]:
# 将各列空列表转为None
# list如果空列表，if x 时返回的是False
for col in df_all.columns:
    df_all[col] = df_all[col].map(lambda x:x if x else None)

### 去掉部分字段为空的数据

In [13]:
# 查看各列的空值情况
for col in df_all.columns:
    print("----------------")
    print(df_all[col].isnull().value_counts())

----------------
False    126711
Name: _id, dtype: int64
----------------
False    99136
True     27575
Name: actor, dtype: int64
----------------
False    106360
True      20351
Name: date, dtype: int64
----------------
False    118695
True       8016
Name: director, dtype: int64
----------------
False    126711
Name: id, dtype: int64
----------------
False    121569
True       5142
Name: language, dtype: int64
----------------
True     85065
False    41646
Name: rate, dtype: int64
----------------
True     85065
False    41646
Name: rating_num, dtype: int64
----------------
False    126395
True        316
Name: region, dtype: int64
----------------
False    85622
True     41089
Name: runtime, dtype: int64
----------------
False    126711
Name: title, dtype: int64
----------------
False    126711
Name: type, dtype: int64
----------------
False    126364
True        347
Name: year, dtype: int64


如果以下的数据出现空值，则整行去掉：

* 演员 actor
* 上映时间 date
* 导演 director
* 语言 language
* 类型 type
* 地区 region

In [14]:
df_all1 = df_all.loc[~(df_all['actor'].isnull() | df_all['date'].isnull() | 
                       df_all['director'].isnull()|df_all['language'].isnull()|
                       df_all['type'].isnull() | df_all['region'].isnull() )]

In [15]:
df_all1.shape

(82564, 13)

In [16]:
df_all1.head(1)

,_id,actor,date,director,id,language,rate,rating_num,region,runtime,title,type,year
0,5c024433f625b7c00bfbe877,"[扎克·埃夫隆, 莱斯利·曼恩, 托马斯·列农, 马修·派瑞, Tyler Steelman...",[2009-04-17(美国)],[布尔·斯蒂尔斯],2213591,[英语],7.3,155828,[美国],[102 分钟],重返十七岁 17 Again,"[剧情, 喜剧, 爱情]",2009


### 填充上映年份

有些电影有上映日期，但年份为空，下面填充这一部分数据

In [17]:
# 查看这一部分的数据
df = df_all1.loc[df_all1['year'].isnull() & df_all1['date'].notnull()]
df

,_id,actor,date,director,id,language,rate,rating_num,region,runtime,title,type,year
1263,5c02453bf625b7c00bfc1f04,"[松平哲郎, 品川美月, 松永美規, みぶ真也, 浅尾典彦, 宮瀬れい]",[2010-07-23],[阿見松ノ介],5273374,[日语],None,None,[日本],None,心灵音 The Movie 心霊音 THE MOVIE,[恐怖],None
1546,5c024543f625b7c00bfc213c,"[Christian Campbell, Sarah Chalke, Carly Pope]",[1998-03-22],[Christopher Leitch],1298504,[英语],None,None,[美国],None,等着你回来 I've Been Waiting for You (TV),"[恐怖, 惊栗]",None
4417,5c0246f5f625b7c00bfc676e,"[羽田圭子, 羽田圭子, 雨宮夕子]",[1997-02-28(日本)],[伊藤正治],26947984,[日语],None,None,[日本],None,生保の女 それぞれの事情,"[剧情, 情色]",None
7490,5c024aa2f625b7c00bfcb5a3,"[欧阳震华, 李铭顺, 张世, 徐峥, 寇振海, 保剑锋, 宁静, 范文芳, 牛萌萌, 唐一...",[2005],[刘志],1830531,[普通话],6.4,724,[中国大陆],None,福禄寿三星报喜,[剧情],None
8467,5c024b52f625b7c00bfcdb69,"[笠原弘子, 坂本千夏, 荒川太郎, 渡辺真砂子, 椎名碧流, 勝田治美]",[1993-08-14],[难波日登志],5349437,[日语],None,None,[日本],None,3丁目のタマ おねがい!モモちゃんを捜して!!,"[动画, 儿童]",None
8520,5c024b58f625b7c00bfcddb4,"[小林裕介, 坂本真绫, 村中知, Lynn, 木村珠莉, 河西健吾, 福岛润, 水树奈奈,...",[2016-12-10],[江崎慎平],26933232,[日语],None,None,[日本],[103分钟],怪物弹珠 THE MOVIE モンスターストライク THE MOVIE はじまりの場所へ,"[剧情, 动作, 动画, 儿童, 奇幻, 冒险]",None
8609,5c024b62f625b7c00bfce28a,[Mel Blanc],[1979-11-27],"[Friz Freleng, Chuck Jones]",3545184,[英语],None,None,[美国],None,Bugs Bunny's Looney Christmas Tales,"[喜剧, 动画, 家庭]",None
8705,5c024b6af625b7c00bfce7d7,[Mel Blanc],[1980-04-01],[Friz Freleng],3431696,[英语],None,None,[美国],None,Daffy Ducks Easter Show,[动画],None
8751,5c024b70f625b7c00bfceac6,"[田中真弓, 富山敬, 岛田敏, 小原乃梨子, 坂本千夏]",[1983],[笹川博],5162570,[日语],None,None,[日本],None,时间飞船系列第七部 イタダキマン,[动画],None
8783,5c024b73f625b7c00bfcebf8,"[德本恭敏, 千叶纱子, 佐藤せつじ, 长岛雄一, 小林沙苗, 细井治, 川中子雅人, 小松...",[2007],[Mamoru Kanbe],5176755,[日语],None,None,[日本],None,鬼公子 炎魔：炎魔 Demon Prince Enma: Enma,[动画],None


In [18]:
df['year'] = df['date'].map(lambda x:x[0][:4])

/Users/yeungshikam/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
df['year'].isnull().value_counts()

False    224
Name: year, dtype: int64

In [92]:
# 原来的部分
df0 = df_all1.loc[~(df_all1['year'].isnull() & df_all1['date'].notnull())]
df0.shape

# 修改的部分 df1 
# 两部分重新整合在一起
df_all2 = pd.concat([df0,df])

In [93]:
df_all2.shape

(82564, 13)

In [94]:
# 对index进行重置
df_all2 = df_all2.reset_index(drop=True)

### 数据格式处理

In [95]:
df_all2.head(5)

,_id,actor,date,director,id,language,rate,rating_num,region,runtime,title,type,year
0,5c024433f625b7c00bfbe877,"[扎克·埃夫隆, 莱斯利·曼恩, 托马斯·列农, 马修·派瑞, Tyler Steelman...",[2009-04-17(美国)],[布尔·斯蒂尔斯],2213591,[英语],7.3,155828,[美国],[102 分钟],重返十七岁 17 Again,"[剧情, 喜剧, 爱情]",2009
1,5c024433f625b7c00bfbe87f,"[狄龙, 张国荣, 周润发, 石天, 关山, 朱宝意, 曾江, 成奎安, 吴孟达, 王正方]",[1987-12-17(香港)],[吴宇森],1297862,[粤语 / 汉语普通话 / 英语],8.2,70082,[香港],[105 分钟],英雄本色2,"[剧情, 动作, 犯罪]",1987
2,5c024433f625b7c00bfbe8df,"[朱丽叶·比诺什, 贝努特·里格恩特, 弗罗伦斯·派梅尔, 夏洛特·韦里, 海伦·文森特, ...","[1993-09-08(法国), 1993-10-10(波兰)]",[克日什托夫·基耶斯洛夫斯基],1292048,[法语 / 波兰语 / 罗马尼亚语],8.5,74953,[法国 / 波兰 / 瑞士],[98分钟],蓝白红三部曲之蓝 Trois couleurs: Bleu,"[剧情, 爱情, 音乐]",1993
3,5c024434f625b7c00bfbe974,"[威尔·史密斯, 罗莎里奥·道森, 伍迪·哈里森, 巴里·佩珀, 迈克尔·伊雷, 比尔·斯米...",[2008-12-19],[加布里埃莱·穆奇诺],2969282,[英语 / 西班牙语],8.1,95242,[美国],[123 分钟],七磅 Seven Pounds,[剧情],2008
4,5c024434f625b7c00bfbe9ba,"[阿萨·巴特菲尔德, 伊娃·格林, 塞缪尔·杰克逊, 朱迪·丹奇, 艾拉·珀内尔, 克里斯·...","[2016-12-02(中国大陆), 2016-09-30(美国)]",[蒂姆·波顿],7051830,[英语],7.2,106468,[美国],[127分钟],佩小姐的奇幻城堡 Miss Peregrine's Home for Peculiar Ch...,"[剧情, 奇幻, 冒险]",2016


#### 语言 language

In [96]:
# [粤语 / 汉语普通话 / 英语] 转为 ['粤语','汉语普通话','英语']
df_all2['language'] = df_all2['language'].map(lambda x:"".join(x).split("/"))

In [97]:
# 再去掉list中每个字符串的前后空格
df_all2['language'] = df_all2['language'].map(lambda x :[i.strip() for i in x])

#### 地区 region

In [98]:
# 同语言
df_all2['region'] = df_all2['region'].map(lambda x:"".join(x).split("/"))
df_all2['region'] = df_all2['region'].map(lambda x :[i.strip() for i in x])

In [99]:
region_set = set()
for each in df_all2['region']:
    for i in each:
        region_set.add(i)

In [100]:
region_set

{'(Canada)',
 '(Indonesia)',
 '(Spain)',
 '(Turkey)',
 '1958-06-29',
 'ATV',
 'Afghanistan',
 'Albania',
 'Algeria',
 'America',
 'American',
 'Angola',
 'Argentina',
 'Argentina)',
 'Armania',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'BBC',
 'Bahamas',
 'Bangladesh',
 'Belarus',
 'Belgium',
 'Benin',
 'Bhutan',
 'Bosnia and Herzegovina',
 'Brasil',
 'Brazil',
 'Bulgaria',
 'Burkina Faso',
 'CANADA',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Chile',
 'China',
 'Colombia',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czech Republic',
 'Czech Republic | Canada',
 'Czechoslovakia',
 'Denmark',
 'Dominica',
 'Dominican Republic',
 'East Germany',
 'Egypt',
 'English',
 'Estonia',
 'Federal Republic of Yugoslavia',
 'Finland',
 'France',
 'French',
 'Georgia',
 'Germany',
 'Germany Germany',
 'Ghana',
 'Greece',
 'Greenland',
 'Guatemala',
 'Guinea',
 'Hungary',
 'Iceland',
 'India',
 'Indonesia',
 'Iran',
 'Ireland',
 'Israel',
 'Italy',
 'Jamaica',
 'Japan',
 'Kazakhstan',
 'Keny

In [105]:
# 可以从上面看到同一个国家会有多个表达
# 对于中英混合的表达转换为纯中文，且将繁体转换为中文
def ZhEn2Zh(x):
    pattern = re.compile("[一-龥]+")
    if pattern.match(x):
        x = re.sub("[^一-龥]+","", x)   
    # 繁体转换为简体
    x = langconv.Converter('zh-hans').convert(x)     
    return x

In [107]:
df_all2['region'] = df_all2['region'].map(lambda x: [ZhEn2Zh(i) for i in x])

In [122]:
# 对于英文的表达，选一些主流地区进行转换
region_dict = {"America":"美国","American":"美国","US":"美国","U.S.A":"美国","USA":"美国","usa":"美国",
               "United States USA":"美国","Argentina":"阿根廷","Argentina)":"阿根廷","Mexico":"墨西哥",
               "Canada":"加拿大","CANADA":"加拿大","(Canada)":"加拿大","Brazil":"巴西","Brasil":"巴西",
               "BBC":"英国","UK":"英国","uk":"英国",
               "Australia":"澳大利亚","Austria":"澳大利亚",                             
               "china":"中国","China":"中国","中国杭州":"中国",
               "India":"印度","india":"印度","Japan":"日本","South Korea":"韩国",
               "(Spain)":"西班牙","Spain":"西班牙","Germany":"德国","Germany Germany":"德国",
               "Sweden":"瑞典","sweden":"瑞典","France":"法国","Franch":"法国","Italy":"意大利",
               "Russia":"俄罗斯","Russian":"俄罗斯","Russion":"俄罗斯",
               "俄国":"俄罗斯","俄语":"俄罗斯","Soviet Union":"苏联",
               "印尼":"印度尼西亚","(Indonesia)":"印度尼西亚","Indonesia":"印度尼西亚","indonesia":"印度尼西亚"}

In [123]:
def region_update(x):
    if x in region_dict:
        x = region_dict[x]
    return x

In [124]:
df_all2['region'] = df_all2['region'].map(lambda x: [region_update(i) for i in x])

#### 片长 runtime

In [126]:
df_all2['runtime'] = df_all2['runtime'].map(lambda x:re.findall("\d+",x[0])[0] if x else None)

In [127]:
df_all2['runtime'].head()

0    102
1    105
2     98
3    123
4    127
Name: runtime, dtype: object

#### 评分rate 、评分人数rating_num 年份year

In [128]:
# 更改为数值型
df_all2['rate'] = df_all2['rate'].astype(float)
df_all2['rating_num'] = df_all2['rating_num'].astype(float)
df_all2['year'] = df_all2['year'].astype(int)

### 筛选掉数据爬取时未上映的电影

In [129]:
df_all2.shape

(82564, 13)

In [130]:
# year: 2019
df_all3 = df_all2.loc[~(df_all2['year'] >=2019)]

In [131]:
df_all3.shape

(82287, 13)

In [132]:
df_all3.to_csv("douban-world-moive-data.csv",index=False)